# Analisis de las bases de datos del SII correspondientes a los años 2018 y 2022

#### Analisis de los datos del SII: 2018 y 2022

In [ ]:
import pandas as pd
import datatable as dt
import numpy as np
pd.options.display.max_columns = None
path_año_antiguo = "BRORGA2441_NAC_2018_1/BRORGA2441N_NAC_2018_1"
path_año_antiguo_lc = "BRORGA2441_NAC_2018_1/BRORGA2441NL_NAC_2018_1"
path_año_nuevo = "BRORGA2441_NAC_2022_1/BRORGA2441N_NAC_2022_1"
path_año_nuevo_lc = "BRORGA2441_NAC_2022_1/BRORGA2441NL_NAC_2022_1"
dt_antiguo = dt.fread(path_año_antiguo, encoding = 'latin')
dt_antiguo_lc = dt.fread(path_año_antiguo_lc, encoding = 'latin')
dt_nuevo = dt.fread(path_año_nuevo, encoding = "latin")
dt_nuevo_lc = dt.fread(path_año_nuevo_lc, encoding = 'latin')
#Nota 1: en las bases de datos con las lineas de construccion aparece un rol de destino tipo X, este rol seguramente fue un error de tipeo ya que siempre aparece en la otra base como un rol tipo C y aparte la X esta al lado de la tecla C

In [ ]:
#diccionarios utilizados para remplazar los codigos de las columnas por los nombres y algunos roles
codigo_a_comuna = pd.concat(pd.read_html('https://zeus.sii.cl/avalu_cgi/br/brch10.sh', match = 'Nombre Comuna', header = 0, index_col = 0)[1:3])['Nombre Comuna']
año_antiguo = path_año_antiguo[-6: -2]
año_nuevo = path_año_nuevo[-6: -2]
codigo_a_rol = {
    'A': 'Agrícola',
    'B': 'Agroindustrial',
    'C': 'Comercio',
    'D': 'Deporte y Recreación',
    'E': 'Educación y Cultura',
    'F': 'Forestal',
    'G': 'Hotel, Motel',
    'H': 'Habitacional',
    'I': 'Industria',
    'L': 'Bodega y Almacenaje',
    'M': 'Minería',
    'O': 'Oficina',
    'P': 'Administración Pública y Defensa',
    'Q': 'Culto',
    'S': 'Salud',
    'T': 'Transporte y Telecomunicaciones',
    'V': 'Otros no considerados',
    'W': 'Sitio Eriazo',
    'Y': 'Gallineros, chancheras y otros',
    'Z': 'Estacionamiento'
    }

columnas_n_nac = {
    'C0': 'Comuna',
    'C1': 'numero_manzana',
    'C2': 'numero_predial',
    'C3': 'direccion_o_nombre_del_predio',
    'C6': 'Código_de_destino_principal',
    'C14': 'Superficie_terreno_'}

### Funciones definidas:
##### -superficie_nac: deja solo las columnas C0 (los codigos de las comunas) y C14 (superficie de los terrenos) y las agrupa en torno a las comunas, sumando todo los metros cuadrados.
##### -man_pre: deja las columnas de Comuna, manzana, predio, direccion, codigo de rol y superficie del terreno. La idea es poder buscar la direccion de un predio, rol y superficie a partir de la comuna, manzana y predio.
##### -codigo_lc: permite la visualizacion de un archivo con los codigos de las lineas de contruccion como columnas, con una columna extra con las sumas de las areas de cada comuna.
##### -comparacion_comuna: agrupa las areas de las superficies por comuna y año, agrega una columna con una diferencia entre los años.
##### -comparacion_lc: hay 2 opciones:
##### -1) negativos = False: muestra la diferencia entre los años por comuna y rol. 
##### -2) negativos = True: muestra solo las comunas que tienen por lo menos una diferencia en un rol que fue negativo.

In [ ]:
def superficie_nac(data, fecha, nombre = False):
    fecha = 'Superficie_total_' + str(fecha)
    data = data[:, [0, 14]]
    data.names = {"C0" : "Comuna", 'C14': 'Superficie_total_'}
    data = data[:, dt.sum(dt.f.Superficie_total_), dt.by(dt.f.Comuna)]
    data = data.to_pandas()
    data = data.rename(columns = {'Superficie_total_': fecha})
    if nombre == True:
        data['Comuna'] = data['Comuna'].replace(codigo_a_comuna)
    return data

def man_pre(data, fecha, nombre = False, destino = False):
    fecha = 'Superficie_terreno_' + str(fecha)
    data = data[:, [0, 1, 2, 3, 6, 14]]
    data.names = columnas_n_nac
    data.names = {'Superficie_terreno_': fecha}
    data = data.to_pandas()
    if nombre == True:
        data['Comuna'] = data['Comuna'].replace(codigo_a_comuna)
    if destino == True:
        data['Código_de_destino_principal'] = data['Código_de_destino_principal'].replace(codigo_a_rol)
    return data

def codigo_lc(data, año, nombre = False):
    DT = data.copy(deep = True)
    del data
    DT.names = {"C0": "Comuna", 
                     "C7": "superficie_lineas_construccion",
                     "C8": "codigo_destino_linea_construccion"}
    DT = DT[:, ['Comuna','superficie_lineas_construccion','codigo_destino_linea_construccion']]
    DT = DT[:, dt.sum(dt.f.superficie_lineas_construccion), dt.by("Comuna", "codigo_destino_linea_construccion")].to_pandas().replace(to_replace = '', value = 'Otro (Desconocido)')
    DT = DT.set_index('Comuna')
    DT['codigo_destino_linea_construccion'] = DT['codigo_destino_linea_construccion'].replace(to_replace = 'X', value = 'C')
    DT = DT.pivot_table(index = 'Comuna',columns = 'codigo_destino_linea_construccion', values = ['superficie_lineas_construccion'], aggfunc = 'sum').fillna(0).astype(int)
    columnas = DT.columns.tolist()
    cols = columnas[:-6] + columnas[-5:-1]
    cols.append(columnas[-6])
    cols.append(columnas[-1])
    DT = DT[cols].loc[:, 'superficie_lineas_construccion']
    DT.rename(columns = codigo_a_rol, inplace = True)
    DT.columns.name = None
    DT = DT.reset_index()
    DT['suma_predios_' + año] = DT.sum(axis = 1)
    if nombre == True:
        DT['Comuna'] = DT['Comuna'].replace(codigo_a_comuna)
    return DT

def comparacion_comuna(data_1, data_2, a_a, a_n, nombre = False):
    data_cl_1 = superficie_nac(data_1, a_a)
    data_cl_2 = superficie_nac(data_2, a_n)
    df_1_2 = pd.concat([data_cl_1, data_cl_2], axis = 1).iloc[:, [0, 1, 3]]
    df_1_2['diferencia_superficies'] = df_1_2.iloc[:, -1] - df_1_2.iloc[:, -2] 
    if nombre == True:
        df_1_2['Comuna'] = df_1_2['Comuna'].replace(codigo_a_comuna)
    return df_1_2

def comparacion_lc(data_1_lc, data_2_lc, a_a, a_n, nombre = False, negativos = False):
    df_sum = codigo_lc(data_1_lc, a_a, nombre).copy()
    df_negativos = codigo_lc(data_2_lc, a_n, nombre).copy()
    x = 0
    z = 0
    for x in range(1, len(df_negativos.columns)):
        for ind in df_negativos.index:
            diferencia = df_negativos.iloc[ind, x] - df_sum.iloc[ind, x]
            df_sum.iloc[ind, x] = diferencia
            if diferencia < 0:
                df_negativos.iloc[ind, x] = diferencia
            else:
                df_negativos.iloc[ind, x] = 0
    df_sum['diferencia_suma_de_codigos'] = df_sum.iloc[:, -1]
    df_sum.drop(columns = 'suma_predios_2018', inplace = True)
    df_sum.rename(columns = {'comuna': 'Comuna'}, inplace = True)
    if nombre == True:
        df_sum['Comuna'] = df_sum['Comuna'].replace(codigo_a_comuna)
    if negativos == True:
        rows_drop = []
        for ind in df_negativos.index:
            for z in range(1, len(df_negativos.columns)):
                celda = df_negativos.iloc[ind, z]
                if celda != 0:
                    rows_drop.append(ind)
                    continue
        rows_drop = list(dict.fromkeys(rows_drop))
        resultado = df_negativos.iloc[rows_drop, :]
        return resultado
    return df_sum


### informacion de un predio a partir de la comuna, numero de manzana y numero predial

In [ ]:
df1 = man_pre(dt_antiguo, año_antiguo, nombre = True, destino = True)
df1

In [ ]:
if type(df1['Comuna'][0]) == np.intc :
    comuna = int(input('Codigo de la comuna a comparar: '))
elif type(df1['Comuna'][0]) == str :
    comuna = input('Nombre de la comuna a comparar: ').upper()
manzana = int(input('Ingresa un numero de manzana: '))
predial = int(input('Ingresa un numero de predio: '))
tdf = df1.loc[df1.Comuna == comuna, :]
tdf = tdf.loc[tdf.numero_manzana == manzana]
display(tdf.loc[tdf.numero_predial == predial, :])

In [ ]:
df2 = man_pre(dt_nuevo, año_nuevo, nombre = True, destino = True)
df2

In [ ]:
if type(df2['Comuna'][0]) == np.intc :
    comuna = int(input('Codigo de la comuna a comparar: '))
elif type(df2['Comuna'][0]) == str :
    comuna = input('Nombre de la comuna a comparar: ').upper()
manzana = int(input('Ingresa un numero de manzana: '))
predial = int(input('Ingresa un numero de predio: '))
tdf = df2.loc[df2.Comuna == comuna, :]
tdf = tdf.loc[tdf.numero_manzana == manzana]
display(tdf.loc[tdf.numero_predial == predial, :])

### Suma de las superficies construidas por comuna
#### areas en metros cuadrados 

In [ ]:
df_union = comparacion_comuna(dt_antiguo, dt_nuevo, año_antiguo, año_nuevo, nombre = True)
df_union

In [ ]:
display(df_union.loc[df_union.diferencia_superficies < 0, :]) 
#comunas con las diferencias negativas, osea que con el paso del tiempo disminuyerin su superficie contruida entre los años 2018 y 2022

#### Comparacion de las superficies construidas

In [ ]:
if type(df_union['Comuna'][0]) == np.intc :
    comuna = int(input('Codigo de la comuna a comparar: '))
    tipo = 'la comuna de codigo'
elif type(df_union['Comuna'][0]) == str :
    comuna = input('Nombre de la comuna a comparar: ').upper()
    tipo = 'comuna'
df_1 = df_union.iloc[:, [-4, -3]]
df_2 = df_union.iloc[:, [-4, -2]]
df_1 = df_1.loc[df_1.Comuna == comuna].iloc[0][-1]
df_2 = df_2.loc[df_2.Comuna == comuna].iloc[0][-1]
print(f'Area total de {tipo} {comuna} para la base de datos del año {año_antiguo}: {df_1} m^2')
print(f'Area total de la {tipo} {comuna} para la base de datos del año {año_nuevo}: {df_2} m^2')
print(f'Diferencia entre las areas entre el {año_antiguo} y el {año_nuevo}: {df_2 - df_1} m^2')
print(f'Porcentaje de aumento: {(df_2 - df_1)/df_1 * 100}%')
if ((df_2 - df_1)/df_1 * 100) < 0:
    print(f'{tipo.upper()} {comuna} DISMINUYO SU SUPERFICIE ENTRE LOS AÑOS {año_antiguo} y {año_nuevo}')

### Comparacion de los roles de construccion y lineas de construccion

In [ ]:
data_2018 = codigo_lc(dt_antiguo_lc, año_antiguo).set_index('Comuna').join(superficie_nac(dt_antiguo, año_antiguo).set_index('Comuna')).reset_index()
display(data_2018) #dataframe con la cantidad de metros cuadrados construidos por comuna y rol en el año 2018
data_2022 = codigo_lc(dt_nuevo_lc, año_nuevo).set_index('Comuna').join(superficie_nac(dt_nuevo, año_nuevo).set_index('Comuna')).reset_index()
display(data_2022) #dataframe con la cantidad de metros cuadrados construidos por comuna y rol en el año 2022

In [ ]:
df_comparacion = comparacion_lc(dt_antiguo_lc, dt_nuevo_lc, año_antiguo, año_nuevo, nombre = True)
df_comparacion #dataframe con las diferencias de area entre el 2022 y 2018, tambien por comuna y rol

In [ ]:
df_negativos = comparacion_lc(dt_antiguo_lc, dt_nuevo_lc, año_antiguo, año_nuevo, nombre = True, negativos = True)
df_negativos #Dataframe con todas las comunas que tuvieron por lo menos un rol que fue negativo (osea que del año 2018 al 2022 disminuyo la cantidad construida para un determinado rol)